# Fonction native *sorted*

## 1. Problématique

Comparons les performances du tri par insertion à celles de la fonction native *sorted*.

In [1]:
from time import time

def duree_tri(fonction, t: tuple)->tuple:
    deb = time()
    fonction(list(t))
    fin = time()
    return (fonction.__name__, fin-deb)

def tri_insertion(tab):
    taille = len(tab)
    for i in range(1,taille):
        en_cours = tab[i]
        j = i-1
        while j >= 0 and tab[j] > en_cours:
            tab[j+1] = tab[j]
            j -= 1
        tab[j+1] = en_cours
    return tab

In [2]:
from random import randint

l = tuple(randint(0,100) for _ in range(10000))
print(duree_tri(tri_insertion, l))

('tri_insertion', 4.343232870101929)


In [3]:
print(duree_tri(sorted, l))

('sorted', 0.0029120445251464844)


### Quel algorithme de tri est implémenté dans la fonction *sorted*?

## 2. Nouvelle approche

### 2.1 Résoudre de petits problèmes...
Une liste de un élément est triée.

<table>
    <tr>
        <td>8</td><td class="vide"> </td><td>5</td><td class="vide"> </td><td>4</td><td class="vide"> </td><td>7</td><td class="vide"> </td><td>9</td><td class="vide"> </td><td>6</td><td class="vide"> </td><td>3</td>
    </tr>
</table>
    

fusionner(liste gauche, liste droite)
<table>
    <tr>
        <td>5</td><td>8</td><td class="vide"> </td><td>4</td><td>7</td><td class="vide"> </td><td>6</td><td>9</td><td class="vide"> </td><td>3</td>
    </tr>
</table>
    

fusionner(liste gauche, liste droite)
<table>
    <tr>
        <td>4</td><td>5</td><td>7</td><td>8</td><td class="vide"> </td><td>3</td><td>6</td><td>9</td>
    </tr>
</table>
    

fusionner(liste gauche, liste droite)
<table>
    <tr>
        <td>3</td><td>4</td><td>5</td><td>6</td><td>7</td><td>8</td><td>9</td>
    </tr>
</table>
    

### 2.2 ... Pour solutionner de gros problèmes
Décomposons un gros problèmes en petits.

<table>
    <tr>
        <td>8</td><td>5</td><td>4</td><td>7</td><td>9</td><td>6</td><td>3</td>
    </tr>
</table>
    

décomposer en deux listes et recommencer sur ces deux listes
<table>
    <tr>
        <td>8</td><td>5</td><td>4</td><td>7</td><td class="vide"> </td><td>9</td><td>6</td><td>3</td>
    </tr>
</table>    

décomposer en deux listes et recommencer sur ces deux listes
<table>
    <tr>
        <td>8</td><td>5</td><td class="vide"> </td><td>4</td><td>7</td><td class="vide"> </td><td>9</td><td>6</td><td class="vide"> </td><td>3</td>
    </tr>
</table>
    

décomposer en deux listes et recommencer sur ces deux listes
<table>
    <tr>
        <td>8</td><td class="vide"> </td><td>5</td><td class="vide"> </td><td>4</td><td class="vide"> </td><td>7</td><td class="vide"> </td><td>9</td><td class="vide"> </td><td>6</td><td class="vide"> </td><td>3</td>
    </tr>
</table>
    

### 2.3 Diviser pour régner: un algorithme récursif
- On appelle récursivement la fonction de tri pour décomposer en sous-listes.
- On fusionne les sous-listes en remontant la pile d'appel.


$$
tri(liste) = \left\{
    \begin{array}{ll}
        liste & \mbox{si la taille de liste} \leq 1 \\
        fusionner(\mbox{tri(liste gauche), tri(liste droite))} & \mbox{sinon }
    \end{array}
\right.
$$

### 2.4 L'étape de fusion

**Activité 1:** Écrire une fonction $fusionner(gauche: list, droite: list)\;\rightarrow\;list$ qui renvoie une liste triée composée des éléments de *gauche* et *droite* déjà triées.

Correction

In [6]:
def fusionner(gauche: list, droite: list)-> list:
    res = []
    i,j = 0,0
    while i < len(gauche) and j < len(droite):
        if droite[j] < gauche[i]:
            res.append(droite[j])
            j += 1
        else:
            res.append(gauche[i])
            i += 1
            
    #ajout de la fin de liste restante
    if i == len(gauche):
        res.extend(droite[j:])
    if j == len(droite):
        res.extend(gauche[i:])
    return res

In [8]:
gauche = [3,4,8,15]
droite = [1,2,6,9]
fusionner(gauche,droite)

[1, 2, 3, 4, 6, 8, 9, 15]

In [9]:
def fusionner2(gauche: list,droite: list)->list:
    res = []
    while gauche and droite:
        if gauche[0] < droite[0]:
            res.append(gauche.pop(0))
        else:
            res.append(droite.pop(0))
            
    #ajout de la fin de liste restante
    if gauche:
        res.extend(gauche)
    if droite:
        res.extend(droite)
    return res

In [10]:
gauche = [3,4,8,15]
droite = [1,2,6,9]
fusionner2(gauche,droite)

[1, 2, 3, 4, 6, 8, 9, 15]

### 2.5 Le code complet du tri fusion

$$
tri(liste) = \left\{
    \begin{array}{ll}
        liste & \mbox{si la taille de liste} \leq 1 \\
        fusionner(\mbox{tri(liste gauche), tri(liste droite))} & \mbox{sinon }
    \end{array}
\right.
$$

In [11]:
def tri_fusion(tab):
    taille = len(tab)
    if taille <= 1:
        return tab
    else:
        milieu = taille//2
        gauche = tri_fusion(tab[:milieu])
        droite = tri_fusion(tab[milieu:])
        return fusionner(gauche,droite)